In [ ]:
import net3d_3_v11_5 as n3
import functions_py3 as fc

import json
jlo = lambda s: json.load(open(s,'r'))
%pylab inline
%config InlineBackend.figure_format = 'retina'
from itertools import combinations
import os
from scipy import interpolate
import time 

In [ ]:
## measure elatic energy for straight-link mouse brain network, relax to ensure no link overlap ##

params = {
    'links':
                   {'k':1.0,'amplitude':1e2, 'thickness': 0.14067288590231117, 
                    'Temp0':.0, 'ce':10, 'segs':80,
                   'weighted': 1,
                   },
}
net = n3.networkBase(JSON='./mouse_straight_edges_new.json', 
                     fixed=True, keep_paths=True, **params)
net.net_name = 'mouse_straight'

n3.iter_converge(net, its = 500, max_its=1000, save_path='../sims/relax/', draw = 1, verbose = False, tol = 0.00001, c0 = 0.1)
n3.iter_converge(net, its = 500, max_its=3000, save_path='../sims/relax/', draw = 1, verbose = False, tol = 0.00001, ct = 0.01)

net1 = jlo('../sims/relax/mouse_straight.json')
elastic = []
k = net1['info']['links']['k']*79
for i in net1['links']:
    points = array(net1['links'][i]['points'])
    r = net1['links'][i]['radius']
    elastic.append(((points[1:]- points[:-1])**2).sum()*r*k)
elastic_energy = sum(elastic)

open('../sims/stats/mouse_straight.txt','a').write('%f\n'%elastic_energy)

In [ ]:
## measure elastic energy for the real mouse brain network layout, relaxed ##

params = {
    'links':
                   {'k':1.0,'amplitude':1e2, 'thickness': 0.14067288590231117,  
                    'Temp0':.0, 'ce':10, 'segs':80,
                   'weighted': 1,
                   },
}
net = n3.networkBase(JSON='./mouse_brain_links_not_reversed-2019-07-05-symmetrized-line_spreads-aggregated_nodes-n26-L472.json', 
                     fixed=True, keep_paths=True, **params)
net.net_name = 'mouse_real_1'

n3.iter_converge(net, its = 500, max_its=1000, save_path='../sims/relax/', draw = 1, verbose = False, tol = 0.00001, c0 = 0.1)
n3.iter_converge(net, its = 500, max_its=4000, save_path='../sims/relax/', draw = 1, verbose = False, tol = 0.00001, ct = 0.05)

net1 = jlo('../sims/relax/mouse_real_1.json')
elastic = []
k = net1['info']['links']['k']*79
for i in net1['links']:
    points = array(net1['links'][i]['points'])
    r = net1['links'][i]['radius']
    elastic.append(((points[1:]- points[:-1])**2).sum()*r*k)
elastic_energy = sum(elastic)

open('../sims/stats/mouse_real.txt','a').write('%f\n'%elastic_energy)

In [42]:
## measure elastic energy for mouse brain networks at three different temperatures ##

gpu = 0

for i in range(40):
    os.system("""python ./mouse_all.py %d %d %d %d & """ %(i*10, (i+1)*10, gpu, 0))
    time.sleep(1400)
    
for i in range(40):
    os.system("""python ./mouse_all.py %d %d %d %d & """ %(i*10, (i+1)*10, gpu, 1))
    time.sleep(1800)
    
for i in range(40):
    os.system("""python ./mouse_all.py %d %d %d %d & """ %(i*10+600, (i+1)*10+600, gpu, 2))
    time.sleep(1800)

In [55]:
## load results ##

el_sim = []
eps=7.067543494682078
with open('../sims/stats/mouse_sim_relaxed_eps%f_elastic_energy_interpolate_new.txt'%eps, 'r') as f:
    for line in f:
        el_sim.append(float(line.strip()))
        
el_sim_low = []
eps=2.9451562666948146
with open('../sims/stats/mouse_sim_relaxed_eps%f_elastic_energy_interpolate_new.txt'%eps, 'r') as f:
    for line in f:
        el_sim_low.append(float(line.strip()))

el_sim_high = []
eps=14.988894647507053
with open('../sims/stats/mouse_sim_relaxed_eps%f_elastic_energy_interpolate_new.txt'%eps, 'r') as f:
    for line in f:
        el_sim_high.append(float(line.strip()))
        
el_straight = []
with open('../sims/stats/mouse_straight.txt'%eps, 'r') as f:
    for line in f:
        el_straight.append(float(line.strip()))

el_real = []
with open('../sims/stats/mouse_real.txt'%eps, 'r') as f:
    for line in f:
        el_real.append(float(line.strip()))




In [ ]:
## make plot ##

x, y = fc.get_binning(el_sim,12)
x1, y1 = fc.get_binning(el_sim_high,12)
x2, y2 = fc.get_binning(el_sim_low,12)
figure(figsize(10,5))
plot([mean(el_straight)]*2, [0,0.55], '-o', label='$\\widetilde{T}_{min} = 0.38$')
plot(x2, y2, '-o',label='$\\widetilde{T} = 0.43$')
plot(x, y, '-o',label='$\\widetilde{T}_B = 0.49$')
plot([mean(el_real)]*2, [0,0.38],'--',label='empirical mouse brain',linewidth = 3)
plot(x1, y1, '-o',label='$\\widetilde{T} = 0.59$')


xlabel('E',fontsize=20) #, x = 0.8, y = 0)
ylabel('$P(E)$',fontsize=20) #, x = 0, y = 0.36)
xticks(fontsize=15)
yticks([0.0,0.1,0.2,0.3,0.4,0.55],['0.00','0.10','0.20','0.30','0.40','1.00'],fontsize=15)
savefig('../sims/figs/energy_distr.svg',bbox_inches='tight')